In [1]:
import requests
import pytz
from datetime import datetime, timedelta
import time
import csv
import schedule
import threading


current_day_start = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)


def convert_utc_to_local(utc_time_str, api_timezone):
    try:
        utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M')
    except ValueError:
        try:
            utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            try:
                utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S %Z')
            except ValueError:
                try:
                    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M:%S%z')
                except ValueError:
                    try:
                        utc_time = datetime.strptime(utc_time_str, '%Y-%m-%d %H:%M %Z%z')
                    except ValueError:
                        # Ako se string ne pokalapa ni sa jednim formatom
                        print("Nevalidni UTC vremenski format.")
                        return None
    
    utc_timezone = pytz.timezone('UTC')
    local_timezone = pytz.timezone(api_timezone)
    
    # Postavljanje vremenske zone na UTC
    utc_time = utc_timezone.localize(utc_time)
    
    # Pretvaranje u lokalno vrijeme
    local_time = utc_time.astimezone(local_timezone)
    
    # Ako je API vratio vremensku zonu kao dio vremenskog stringa, provjerava jesu li vremenske zone usklađene
    if '%z' in utc_time_str:
        if local_time.strftime('%z') != utc_time.strftime('%z'):
            print("Upozorenje: Nepodudaranje informacija o vremenskim zonama.")
    
    return local_time.strftime('%Y-%m-%d %H:%M:%S')



def get_energy_data(api_key, city='Sarajevo', country='BA'):
    # Postavljamo tajmer za sljedeći poziv
    threading.Timer(180, get_energy_data, args=(api_key, city, country)).start()
    time_str = datetime.now().strftime('%H:%M:%S')

    global current_day_start  # Dodajemo globalnu varijablu

    api_url = 'https://api.weatherbit.io/v2.0/current'


    current_datetime = datetime.now()
    

    params = {
        'city': city,
        'country': country,
        'key': api_key,
    }

    # Popis podataka koje želimo pohraniti
    desired_data = ['datetime','Time','city_name', 'sunrise','sunset','pod','dhi', 'dni','ghi', 'solar_rad','elev_angle','h_angle','uv','temp','app_temp','rh','dewpt','clouds','vis','precip','snow','pres','wind_spd','wind_dir']


    
    try:

        response = requests.get(api_url, params=params)

        if response.status_code == 200:
            data = response.json()


            # Mapiranje engleskih naziva kolona na bosanske nazive
            translation_map = {
                'datetime': 'Datum i vrijeme',
                'Time': 'Vrijeme',
                'city_name': 'Naziv grada',
                'sunrise': 'Izlazak sunca (UTC)',
                'sunset': 'Zalazak sunca (UTC)',
                'pod': 'Dio dana (d = dan / n = noc)',
                'dhi': 'Difuzno horizontalno suncevo zracenje (W/m^2)',
                'dni': 'Direktno normalno suncevo zracenje (W/m^2)',
                'ghi': 'Globalno horizontalno suncevo zracenje (W/m^2)',
                'solar_rad': 'Procijenjeno_suncevo_zracenje',
                'elev_angle': 'Solarni elevacioni ugao (stepeni)',
                'h_angle': 'Solarni satni ugao (stepeni)',
                'uv': 'UV indeks',
                'temp': 'Temperatura',
                'app_temp': 'Aparentna temperatura (°C)',
                'rh': 'Relativna vlaznost (%)',
                'dewpt': 'Temperatura rose (°C)',
                'clouds': 'Oblacnost (%)',
                'vis': 'Vidljivost (km)',
                'precip': 'Padavine (mm/h)',
                'snow': 'Snijeg (mm/h)',
                'pres': 'Pritisak (mb)',
                'wind_spd': 'Brzina vjetra (m/s)',
                'wind_dir': 'Smjer vjetra (stepeni)'
            }

            # Otvori CSV fajl za pisanje
            with open('podaci.csv', mode='a', newline='') as csv_file:
                writer = csv.DictWriter(csv_file, fieldnames=translation_map.keys())

                # Ako je fajl prazan, napiši zaglavlje
                if csv_file.tell() == 0:
                    # Prevedi engleske nazive kolona u bosanske i piši zaglavlje
                    writer.writerow(translation_map)
                
                # Ažuriraj trenutno vrijeme
                current_datetime = datetime.now()

                # Provjeri da li se promjenio datum
                if current_datetime.date() != current_day_start.date():
                    print(f"Novi dan! ")
                    # Update current_day_start
                    current_day_start = datetime(current_datetime.year, current_datetime.month, current_datetime.day, 0, 0, 0)
                
                # Pisanje željenih podataka u CSV datoteku
                for entry in data['data']:
                    # Dinamičko postavljanje vremena na trenutno vrijeme
                    entry['Time'] = time_str
                     # Dodaje red u CSV fajl
                    writer.writerow({key: entry[key] for key in desired_data})

                    #Pretvorba UTC u lokalno vrijeme
                    utc_time_str = entry.get('ob_time')  # Pretpostavka da 'ob_time' sadrži UTC vrijeme
                    api_timezone = 'Europe/Sarajevo'  # Pretpostavka da API vraća podatke u vremenskoj zoni Europe/Sarajevo
                    local_time_str = convert_utc_to_local(utc_time_str, api_timezone)

                    # Printa poruku prije svakog mjerenja
                    print(f"Mjerenje za {datetime.now().date()} u {time_str}.")

                   # Isprintaj sve parametre za svaku opservaciju
                    print(f"Vremenska zona: {entry.get('timezone')}")
                    print(f"Vrijeme posljednje opservacije: {local_time_str}")
                    print(f"Naziv grada: {entry.get('city_name')}")
                    print(f"Geografska sirina: {entry.get('lat')}")
                    print(f"Geografska duzina: {entry.get('lon')}")
                    print(f"Vrijeme izlaska sunca (UTC): {entry.get('sunrise')}")
                    print(f"Vrijeme zalaska sunca (UTC): {entry.get('sunset')}")
                    print(f"Pritisak: {entry.get('pres')} mb")
                    print(f"Pritisak na nivou mora: {entry.get('slp')} mb")
                    print(f"Brzina vjetra: {entry.get('wind_spd')} m/s")
                    print(f"Brzina vjetra (naleti): {entry.get('gust')} m/s")
                    print(f"Smijer vetra: {entry.get('wind_dir')} stepeni")
                    print(f"Puni opis smijera vjetra: {entry.get('wind_cdir_full')}")
                    print(f"Temperatura: {entry.get('temp')} °C")
                    print(f"Aparentna temperatura: {entry.get('app_temp')} °C")
                    print(f"Relativna vlaznost: {entry.get('rh')} %")
                    print(f"Tacka rose: {entry.get('dewpt')} °C")
                    print(f"Oblacnost: {entry.get('clouds')} %")
                    print(f"Dio dana: {entry.get('pod')} (d = dan / n = noć)")
                    print(f"Vidljivost: {entry.get('vis')} km")
                    print(f"Padavine: {entry.get('precip')} mm/h")
                    print(f"Snijeg: {entry.get('snow')} mm/h")
                    print(f"UV indeks: {entry.get('uv')}")
                    print(f"Indeks kvaliteta vazduha (AQI): {entry.get('aqi')}")
                    print(f"Difuzno horizontalno suncevo zracenje: {entry.get('dhi')} W/m^2")
                    print(f"Direktno normalno sunčevo zracenje: {entry.get('dni')} W/m^2")
                    print(f"Globalno horizontalno sunčevo zracenje: {entry.get('ghi')} W/m^2")
                    print(f"Procenjeno suncevo zracenje: {entry.get('solar_rad')} W/m^2")
                    print(f"Solarni elevacioni ugao: {entry.get('elev_angle')} stepeni")
                    print(f"Solarni satni ugao (zastarjelo): {entry.get('h_angle')} stepeni")

                    print("\n" + "-"*40 + "\n")

        else:
            print(f'Error {response.status_code}: {response.text}')

    except Exception as e:
        print(f'Error: {e}')


# Pokrećemo prvi poziv odmah
get_energy_data('994f535cd56f452aade060180c6eda3b')

# Održavamo program aktivnim
while True:
    pass



Mjerenje za 2024-03-02 u 19:55:07.
Vremenska zona: Europe/Sarajevo
Vrijeme posljednje opservacije: 2024-03-02 19:25:00
Naziv grada: Sarajevo
Geografska sirina: 43.84864
Geografska duzina: 18.35644
Vrijeme izlaska sunca (UTC): 05:19
Vrijeme zalaska sunca (UTC): 16:38
Pritisak: 951 mb
Pritisak na nivou mora: 1011 mb
Brzina vjetra: 1.5 m/s
Brzina vjetra (naleti): None m/s
Smijer vetra: 60 stepeni
Puni opis smijera vjetra: east-northeast
Temperatura: 12 °C
Aparentna temperatura: 12 °C
Relativna vlaznost: 58 %
Tacka rose: 4 °C
Oblacnost: 85 %
Dio dana: n (d = dan / n = noć)
Vidljivost: 16 km
Padavine: 0.5 mm/h
Snijeg: 0 mm/h
UV indeks: 0
Indeks kvaliteta vazduha (AQI): 74
Difuzno horizontalno suncevo zracenje: 0 W/m^2
Direktno normalno sunčevo zracenje: 0 W/m^2
Globalno horizontalno sunčevo zracenje: 0 W/m^2
Procenjeno suncevo zracenje: 0 W/m^2
Solarni elevacioni ugao: -15.78 stepeni
Solarni satni ugao (zastarjelo): -90 stepeni

----------------------------------------

Mjerenje za 2024-03-